In [91]:
!pip install scikit-learn
!pip install xgboost
!pip install mlxtend
!pip install lightgbm


In [92]:
import warnings
from sklearn.model_selection import cross_val_predict
import pandas as pd
import numpy as np
import six
import sys
sys.modules['sklearn.externals.six'] = six


warnings.filterwarnings('ignore')


Load Data

In [93]:
train_X = pd.read_csv('preprocessing_train_x.csv')
test_X = pd.read_csv('preprocessing_test_x.csv')
train_df = pd.read_csv('train.csv')


In [94]:
train_X


,Unnamed: 0,product_code,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2
0,0,0.0,2.0,9,5,7,8,4
1,1,0.0,2.0,9,5,14,3,3
2,2,0.0,2.0,9,5,12,1,5
3,3,0.0,2.0,9,5,13,2,6
4,4,0.0,2.0,9,5,9,2,8
...,...,...,...,...,...,...,...,...
26565,26565,4.0,1.0,6,9,6,16,4
26566,26566,4.0,1.0,6,9,10,12,8
26567,26567,4.0,1.0,6,9,1,10,1
26568,26568,4.0,1.0,6,9,2,9,4


In [95]:
y = train_df.failure
y


0        0
1        0
2        0
3        0
4        0
        ..
26565    0
26566    0
26567    0
26568    0
26569    0
Name: failure, Length: 26570, dtype: int64

Model Training

Decision Tree

In [96]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(train_X, y)
dt_pred = dt.predict(test_X)


Random Forest

In [97]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_X, y)
rf_pred = rf.predict(test_X)


Histogram-based Gradient Boost

In [98]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(max_iter=100)
hgb.fit(train_X, y)
hgb_pred = hgb.predict(test_X)


XGBoost

In [99]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(train_X, y)
xgb_pred = xgb.predict(test_X)


LGBM

In [100]:
import re
train_X = train_X.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
test_X = test_X.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))


In [101]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(max_bin=255, n_estimators=100)
lgbm.fit(train_X, y)
lgbm_pred = lgbm.predict(test_X)


AdaBoost

In [102]:
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=50, learning_rate=1.0,
                         algorithm='SAMME.R', random_state=None)
adb.fit(train_X, y)
adb_pred = rf.predict(test_X)


Ensemble

In [103]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[dt, hgb, lgbm, adb, xgb, rf],
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=lr)

sclf.fit(train_X, y)
sclf_pred = sclf.predict(test_X)


Generate submission.csv

In [104]:
# find best result using desicion tree
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['failure'] = (dt_pred)
sample_submission.to_csv('0816091.csv', index=False)
display(sample_submission)


,id,failure
0,26570,1
1,26571,0
2,26572,1
3,26573,1
4,26574,0
...,...,...
20770,47340,0
20771,47341,0
20772,47342,0
20773,47343,0
